In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os
from v4vapp_backend_v2.config.setup import InternalConfig
from pprint import pprint

target_dir = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/"
os.chdir(target_dir)
print("Current working directory:", os.getcwd())


In [ ]:
from v4vapp_backend_v2.accounting.ledger_entries import get_ledger_dataframe
from v4vapp_backend_v2.accounting.balance_sheet import (
    generate_balance_sheet_pandas_from_accounts,
)
from v4vapp_backend_v2.database.db_pymongo import DBConn


CONFIG = InternalConfig(config_filename="devhive.config.yaml").config
db_conn = DBConn()
await db_conn.setup_database()

ledger_df = await get_ledger_dataframe()
balance_sheet_dict = await generate_balance_sheet_pandas_from_accounts(df=ledger_df)


In [ ]:
pprint(ledger_df)

In [ ]:
pprint(balance_sheet_dict)

In [ ]:
from v4vapp_backend_v2.accounting.account_balances import (
    get_account_balance_printout,
    list_all_accounts,
)

all_accounts = await list_all_accounts()
print("All accounts:", all_accounts)
balances = {}
for account in all_accounts:
    print(f"Account: {account}")
    balances[str(account)] = await get_account_balance_printout(
        account=account, line_items=True, df=ledger_df
    )
for account, balance in balances.items():
    print(f"Balance for {account}:")
    print(balance[0])


In [ ]:
from v4vapp_backend_v2.actions.hive_to_lightning import get_verified_hive_client
from nectar.account import Account
from nectar.amount import Amount
from v4vapp_backend_v2.accounting.ledger_account_classes import LedgerAccount
from pprint import pprint

from v4vapp_backend_v2.helpers.crypto_prices import Currency
from math import isclose

print("Checking server and treasury accounts vs Hive balances...")
hive_client, server_name = await get_verified_hive_client()
server_account = Account(server_name, blockchain_instance=hive_client)
treasury_account = Account("devtre.v4vapp", blockchain_instance=hive_client)

# Server account balances
server_ledger_account = LedgerAccount.from_string(
    "Customer Deposits Hive (Asset) - Sub: devser.v4vapp"
)
server_summary_text, server_summary = await get_account_balance_printout(
    account=server_ledger_account,
    line_items=True,
)
print("Server Account Balance Summary:")

pprint("Server Account " + str(server_account.balances.get("available", [])))
if Currency.HIVE.value in server_summary.unit_summaries:
    # print("HIVE Balance:", server_account.balances.get("available", Amount(0, "HIVE"))[0])
    # pprint(server_summary.unit_summaries[Currency.HIVE.value].final_balance)
    if isclose(
        server_summary.unit_summaries[Currency.HIVE.value].final_balance,
        server_account.balances.get("available", Amount(0, "HIVE"))[0],
        abs_tol=0.002,  # absolute tolerance for 3 decimal places
    ):
        print("Balance match for HIVE Server Account")
    else:
        print("Balance mismatch for HIVE Server Account")
        print(
            f"{server_summary.unit_summaries[Currency.HIVE.value].final_balance} != {server_account.balances.get('available', Amount(0, 'HIVE'))[0]}"
        )
if Currency.HBD.value in server_summary.unit_summaries:
    # print("HBD Balance:", server_account.balances.get("available", Amount(0, "HBD"))[1])
    # pprint(server_summary.unit_summaries[Currency.HBD.value].final_balance)
    if isclose(
        server_summary.unit_summaries[Currency.HBD.value].final_balance,
        server_account.balances.get("available", Amount(0, "HBD"))[1],
        abs_tol=0.002,  # absolute tolerance for 3 decimal places
    ):
        print("Balance match for HBD Server Account")
    else:
        print("Balance mismatch for HBD Server Account")


treasury_ledger_account = LedgerAccount.from_string("Treasury Hive (Asset) - Sub: devtre.v4vapp")
# pprint("Treasury Account " + str(treasury_account.balances.get("available", [])))
treasury_summary_text, treasury_summary = await get_account_balance_printout(
    account=treasury_ledger_account,
    line_items=True,
)
print("Treasury Account Balance Summary:")
if Currency.HIVE.value in treasury_summary.unit_summaries:
    # print("HIVE Balance:", treasury_account.balances.get("available", Amount(0, "HIVE"))[0])
    # pprint(treasury_summary.unit_summaries[Currency.HIVE.value].final_balance)
    if isclose(
        treasury_summary.unit_summaries[Currency.HIVE.value].final_balance,
        treasury_account.balances.get("available", Amount(0, "HIVE"))[0],
        abs_tol=0.002,  # absolute tolerance for 3 decimal places
    ):
        print("Balance match for HIVE Treasury Account")
    else:
        print("Balance mismatch for HIVE Treasury Account")
if Currency.HBD.value in treasury_summary.unit_summaries:
    # print("HBD Balance:", treasury_account.balances.get("available", Amount(0, "HBD"))[1])
    # pprint(treasury_summary.unit_summaries[Currency.HBD.value].final_balance)
    if isclose(
        treasury_summary.unit_summaries[Currency.HBD.value].final_balance,
        treasury_account.balances.get("available", Amount(0, "HBD"))[1],
        abs_tol=0.002,  # absolute tolerance for 3 decimal places
    ):
        print("Balance match for HBD Treasury Account")
    else:
        print("Balance mismatch for HBD Treasury Account")

In [ ]:
from datetime import datetime, timezone
from v4vapp_backend_v2.accounting.ledger_entries import get_ledger_entries
from v4vapp_backend_v2.accounting.balance_sheet import (
    balance_sheet_all_currencies_printout,
    balance_sheet_printout,
)
from v4vapp_backend_v2.accounting.profit_and_loss import (
    generate_profit_and_loss_report,
    profit_and_loss_printout,
)

"""
All Balances sheets and all_accounts
"""
balance_sheet_currencies_str = balance_sheet_all_currencies_printout(balance_sheet_dict)
balance_sheet_str = balance_sheet_printout(balance_sheet_dict)


print("Balance sheet (currencies):")
print(balance_sheet_currencies_str)
print("Balance sheet:")
print(balance_sheet_str)

p_and_l = await generate_profit_and_loss_report()
print(profit_and_loss_printout(pl_report=p_and_l))

print("Balances for all accounts:")
for account, (balance, summary) in balances.items():
    print(balance)


print("Ledger entries:")
ledger_entries = await get_ledger_entries(
    collection_name="ledger",
    filter_by_account=None,
    as_of_date=datetime.now(tz=timezone.utc),
)
for ledger_entry in ledger_entries:
    print(ledger_entry)

In [ ]:
for ledger_entry in ledger_entries:
    print(ledger_entry.log_str)

In [ ]:
try:
    print(balances["Customer Liability (Liability) - Sub: v4vapp-test"][0])
except KeyError:
    print("Key not found in balances")


In [ ]:
try:
    print(balances["Customer Deposits Hive (Asset) - Sub: devser.v4vapp"][0])
except KeyError:
    print("Key not found in balances")


In [ ]:
try:
    print(balances["Fee Income Lightning (Revenue) - Sub: umbrel"][0])
except KeyError:
    print("No balance found for 'Fee Income Lightning (Revenue) - Sub: umbrel'")

In [ ]:
try:
    print(balances["Treasury Lightning (Asset) - Sub: umbrel"][1])
except KeyError:
    print("No balance found for 'Treasury Lightning (Asset) - Sub: umbrel'")


In [ ]:
account = LedgerAccount.from_string("Converted Hive Offset (Asset) - Sub: devser.v4vapp")

(printout_str, data) = await get_account_balance_printout(account=account, line_items=True)
print(printout_str)

In [ ]:
ledger_df = await get_ledger_dataframe()
print("Ledger DataFrame:")
print(ledger_df.head())

In [ ]:
balance_sheet_dict = await generate_balance_sheet_pandas_from_accounts()
print("Balance Sheet Dictionary:")
pprint(balance_sheet_dict)

In [ ]:
from v4vapp_backend_v2.accounting.profit_and_loss import (
    generate_profit_and_loss_report,
    profit_and_loss_printout,
)

p_and_l = await generate_profit_and_loss_report()
pprint(p_and_l)

In [ ]:
from v4vapp_backend_v2.accounting.profit_and_loss import (
    generate_profit_and_loss_report,
)

p_and_l_printout = profit_and_loss_printout(
    pl_report=p_and_l,
)

print(p_and_l_printout)

In [ ]:
from v4vapp_backend_v2.accounting.balance_sheet import generate_balance_sheet_pandas_from_accounts

bs = await generate_balance_sheet_pandas_from_accounts()
pprint(bs)

In [ ]:
bs_printout_currencies = balance_sheet_all_currencies_printout(bs)
bs_printout = balance_sheet_printout(bs)
print(bs_printout_currencies)
print(bs_printout)

In [ ]:
from v4vapp_backend_v2.accounting.ledger_entry import LedgerType


account = LedgerAccount.from_string("Customer Deposits Hive (Asset) - Sub: devser.v4vapp")

lightning_spend = await get_ledger_entries(
    filter_by_account=account,
    cust_id="v4vapp-test",
    filter_by_ledger_types=[
        LedgerType.CONV_KEEPSATS_TO_HIVE,
        LedgerType.CONV_HIVE_TO_KEEPSATS,
        LedgerType.CONV_HIVE_TO_LIGHTNING,
        # LedgerType.CONV_LIGHTNING_TO_HIVE,
    ],
)
for entry in lightning_spend:
    print(entry)

lightning_conversion = await get_ledger_entries(
    filter_by_account=account, filter_by_ledger_types=[LedgerType.DEPOSIT_KEEPSATS]
)
for entry in lightning_conversion:
    print(entry)

In [ ]:
# Testing lightning spend limits

from datetime import timedelta

from v4vapp_backend_v2.accounting.account_balances import (
    get_account_lightning_conv,
    check_hive_conversion_limits,
)
from v4vapp_backend_v2.accounting.pipelines.simple_pipelines import (
    filter_sum_credit_debit_pipeline,
)

account = LedgerAccount.from_string("Customer Liability (Liability) - Sub: v4vapp-test")
pipeline = filter_sum_credit_debit_pipeline(
    account=account,
    age=timedelta(minutes=00),
    ledger_types=[
        LedgerType.CONV_HIVE_TO_KEEPSATS,
        LedgerType.CONV_HIVE_TO_LIGHTNING,
        LedgerType.CONV_KEEPSATS_TO_HIVE,
    ],
)
print(pipeline)
collection = InternalConfig.db["ledger"]
cursor = await collection.aggregate(pipeline=pipeline)
async for entry in cursor:
    pprint(entry)

ans = await get_account_lightning_conv(
    cust_id="v4vapp-test",
    age=timedelta(hours=60),
)

print(ans.sats)
ans = await check_hive_conversion_limits("v4vapp-test")
for limit in ans:
    print(limit.output_text)
    print("Limit OK:", limit.limit_ok)
    print("Spend Summary:", limit.conv_summary)
    print("Total Sats:", limit.total_sats)
    print("Total Msats:", limit.total_msats)


limit_ok = all(limit.limit_ok for limit in ans)
print("All limits OK:", limit_ok)

In [ ]:
# Get Keepsats account balances
ans = await check_hive_conversion_limits("v4vapp-test")
pprint(ans)
